###  Pasos a seguir en el proceso de 'scraping':

1. Encuentra la URL que quieres 'escrapear'.
2. Inspecciona la página (código fuente).
3. Localiza los datos que necesitas obtener.
4. Desarrolla tu código en Python.
    1. Crea tu sopa
    2. Busca los elementos que cotienen los datos y extráelos
5. Ejecuta tu código y obten los datos.
6. Alamacena los datos en el formato requerido.

In [1]:
# Importamos librerías
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import html
import numpy as np

## Caso 1: Scraping de un catálogo: Labirratorium

In [2]:
URL = 'https://www.labirratorium.com/es/67-cervezas-por-estilo?page='

Queremos obtener un dataFrame con todas las cervezas del catálogo y sus características descritas. Analizamos la página para ver qué tenemos que hacer para conseguirlo

In [ ]:
# La web tiene 80 páginas con 12 cervezas listadas en cada página.

Hacemos la consulta (request) y creamos la SOPA inicial:

In [3]:
r = requests.get(URL)
soup = bs(r.text, 'lxml')

In [4]:
# Guardamos lista de cervezas
cervezas_grid = soup.find_all(class_='product-image')
cervezas_grid[0]

<div class="product-image">
<!-- @file modules\appagebuilder\views\templates\front\products\file_tpl -->
<ul class="product-flags">
</ul>
<a class="thumbnail product-thumbnail" href="https://www.labirratorium.com/es/inicio/831-arriaca-boris-brew-vikingathor-.html">
<img alt="" class="img-fluid" data-full-size-image-url="https://www.labirratorium.com/27941-large_default/arriaca-boris-brew-vikingathor-.jpg" src="https://www.labirratorium.com/27941-home_default/arriaca-boris-brew-vikingathor-.jpg"/>
<span class="product-additional" data-idproduct="831"></span>
</a>
</div>

In [5]:
# necesitamos acceder a cada una de las cervezas del grid:
lista_URLs = []
for cerveza in cervezas_grid:
    URL_cerveza = cerveza.find('a')['href']
    lista_URLs.append(URL_cerveza)

lista_URLs


['https://www.labirratorium.com/es/inicio/831-arriaca-boris-brew-vikingathor-.html',
 'https://www.labirratorium.com/es/cerveza-lambic/284-boon-kriek-2013.html',
 'https://www.labirratorium.com/es/cervezas-alemanas/225-stortebeker-schwarz-bier.html',
 'https://www.labirratorium.com/es/botella-33cl/199-orval.html',
 'https://www.labirratorium.com/es/cervezas-alemanas/184-augustiner-lagerbier-hell.html',
 'https://www.labirratorium.com/es/inicio/183-schneider-eisbock.html',
 'https://www.labirratorium.com/es/inicio/181-schlenkerla-rauchbier-weizen.html',
 'https://www.labirratorium.com/es/inicio/173-samuel-adams-boston-lager.html',
 'https://www.labirratorium.com/es/inicio/165-laugar-epa.html',
 'https://www.labirratorium.com/es/inicio/82-westmalle-dubbel.html',
 'https://www.labirratorium.com/es/inicio/75-duchesse-de-bourgogne.html',
 'https://www.labirratorium.com/es/inicio/61-tripel-karmeliet33.html']

In [6]:
# Hacemos un nuevo request para la primera cerveza: 
r = requests.get(lista_URLs[0])

# Creamos una sopa específica con la info de cada cerveza
soup_cerveza = bs(r.text, 'lxml')

In [8]:
# Nombre
name = soup_cerveza.find(class_='h1 product-detail-name')
name

<h1 class="h1 product-detail-name" itemprop="name">SanFrutos Vikingathor Boris Brew</h1>

In [10]:
# Precio
price = float(soup_cerveza.find(class_='current-price').find('span')['content'])
price

3.35

In [11]:
# Descripcion corta
descr_short = soup_cerveza.find(class_='description-short').find('p').text
descr_short

'Indian Dunkel'

In [12]:
# Descripción larga
descr_full_list = []

descr_full_p = soup_cerveza.find(class_='product-description').find_all()
for sentence in descr_full_p:
    descr_full_list.append(sentence.text)

descr_full = '\n'.join(descr_full_list)

print(descr_full)

Cerveza de 8.2% ABV intensa, oscura y muy lupulada. Con aroma intenso a maltas tostadas, caramelo, fruta madura, resina e hierbas aromáticas. Delicioso armargor, dulzor equilibrado y sensación de calidez. Refrescante y lupulada. 
Una receta de Boris de Mesones.
Vikingathor ya ha sido galardonada con anterioridad con las Medallas de Plata cosechadas en el European Beer Star de 2011 y 2012 y la Medalla de Plata en el Asia Beer Cup 2013.
Medallas de Plata
European Beer Star
Medalla de Plata
Asia Beer Cup




In [15]:
# Imagen
image = soup_cerveza.find(class_='js-qv-product-cover img-fluid')['src']
image

'https://www.labirratorium.com/27941-large_default/arriaca-boris-brew-vikingathor-.jpg'

In [16]:
 # Brand
brand = soup_cerveza.find(class_='img img-thumbnail manufacturer-logo')['alt']
brand


'Cerveza Sanfrutos'

In [17]:
# Código de barras
try:
    barcode = soup_cerveza.find(class_='product-reference').find('span').text
except:
    barcode = None
print(barcode)

831


In [18]:
# Features
features_dict = {}

try:
    features_dt = soup_cerveza.find_all('dt')
    features_dd = soup_cerveza.find_all('dd')
    for feature, value in zip(features_dt, features_dd):
        features_dict[feature.text] = value.text
except:
    features_dict = {}

features_dict


{'Estilo': 'DUNKEL',
 'Origen': 'Segovia',
 '% Alc.': 'ALTO (6-9%)',
 '% Alc. Exacto': '8.2',
 'Volumen (cl)': '33 Cl',
 'Tipo Fermentación': 'Lager (Baja Fermentación)',
 'IBU': '50-75 Amargor alto',
 'IBU Exacto': '75',
 'Color': 'Marrón',
 'Envase': 'Botella'}

In [19]:
# Creamos un Id único que os permita diferenciar cada entrada en la BBDD
counter = 1
id_cerv = 'lbt_' + str(counter)

id_cerv

'lbt_1'

Ya tenemos todos los datos que queremos de la cerveza: Agrupamos todo en una lista:

In [ ]:
# Agregamos a una lista
lista_cervezas = []

lista_cervezas.append([
    ("id", id_cerv),
    ("name", name),
    ('price', price),
    ('descr_short', descr_short),
    ('descr_full', descr_full),
    ('image', image),
    ('brand', brand),
    ('barcode', barcode),
    ('features', features_dict)
])

lista_cervezas

### Ya sabemos obtener todos los datos que nos interesan de una cerveza, ahora tenemos que aplicar esta lógica para obtener todas las demás 

Construir un DataFrame

Cerveza 1 de 12, pag 1/4
Cerveza 2 de 12, pag 1/4
Cerveza 3 de 12, pag 1/4
Cerveza 4 de 12, pag 1/4
Cerveza 5 de 12, pag 1/4
Cerveza 6 de 12, pag 1/4
Cerveza 7 de 12, pag 1/4
Cerveza 8 de 12, pag 1/4
Cerveza 9 de 12, pag 1/4
Cerveza 10 de 12, pag 1/4
Cerveza 11 de 12, pag 1/4
Cerveza 12 de 12, pag 1/4
Cerveza 1 de 12, pag 2/4
Cerveza 2 de 12, pag 2/4
Cerveza 3 de 12, pag 2/4
Cerveza 4 de 12, pag 2/4
Cerveza 5 de 12, pag 2/4
Cerveza 6 de 12, pag 2/4
Cerveza 7 de 12, pag 2/4
Cerveza 8 de 12, pag 2/4
Cerveza 9 de 12, pag 2/4
Cerveza 10 de 12, pag 2/4
Cerveza 11 de 12, pag 2/4
Cerveza 12 de 12, pag 2/4
Cerveza 1 de 12, pag 3/4
Cerveza 2 de 12, pag 3/4
Cerveza 3 de 12, pag 3/4
Cerveza 4 de 12, pag 3/4
Cerveza 5 de 12, pag 3/4
Cerveza 6 de 12, pag 3/4
Cerveza 7 de 12, pag 3/4
Cerveza 8 de 12, pag 3/4
Cerveza 9 de 12, pag 3/4
Cerveza 10 de 12, pag 3/4
Cerveza 11 de 12, pag 3/4
Cerveza 12 de 12, pag 3/4
Cerveza 1 de 12, pag 4/4
Cerveza 2 de 12, pag 4/4
Cerveza 3 de 12, pag 4/4
Cerveza 4 de 12,

,id_libro,name,price,descr_short,descr_full,brand,barcode,image,features
0,lbt_1,SanFrutos Vikingathor Boris Brew,3.35,Indian Dunkel,"Cerveza de 8.2% ABV intensa, oscura y muy lupu...",Cerveza Sanfrutos,831,https://www.labirratorium.com/27941-large_defa...,"{'Estilo': 'DUNKEL', 'Origen': 'Segovia', '% A..."
1,lbt_1,"Boon Oude Kriek 37,5cl",7.15,Lambic / Kriek,Cerveza de fermentación espontánea (Lambic) de...,Brouwerij F. Boon,284,https://www.labirratorium.com/19351-large_defa...,"{'Estilo': 'KRIEK', 'Origen': 'Bélgica', '% Al..."
2,lbt_1,Störtebeker Schwarz-Bier,2.5,Cerveza negra de baja fermentación,Cerveza negra de baja fermentación de estilo S...,Störtebeker,225,https://www.labirratorium.com/488-large_defaul...,"{'Estilo': 'SCHWARZBIER', 'Origen': 'Alemania'..."
3,lbt_1,Orval,2.95,Belgian Pale Ale,Orval es una cerveza tipo ale y es la que prin...,Orval,199,https://www.labirratorium.com/385-large_defaul...,"{'Estilo': 'BELGIAN PALE ALE', 'Origen': 'Bélg..."
4,lbt_1,Augustiner Lagerbier Hell,2.75,Lager / Helles,Cerveza de baja fermentación de estilo Munich ...,Augustiner,184,https://www.labirratorium.com/367-large_defaul...,"{'Estilo': 'MUNICH HELLES', 'Origen': 'Alemani..."
5,lbt_1,Schneider Aventinus Weizen-Eisbock,2.85,"Cerveza de color medio oscuro, aromas alcohóli...","Cerveza de color medio oscuro, aromas alcohóli...",Schneider,183,https://www.labirratorium.com/366-large_defaul...,"{'Origen': 'Alemania', '% Alc. Exacto': '12', ..."
6,lbt_1,Aecht Schlenkerla Rauchbier Weizen,2.7,,Rauchbier (cerveza ahumada) de trigo con 5.2% ...,Schlenkerla,181,https://www.labirratorium.com/364-large_defaul...,"{'Estilo': 'RAUCHBIER', 'Origen': 'Alemania', ..."
7,lbt_1,Samuel Adams Boston Lager,2.45,Cerveza de baja fermentación (Lager) de color ...,Apariencia: color ambarino cobrizo y espuma pe...,Samuel Adams,173,https://www.labirratorium.com/343-large_defaul...,"{'Estilo': 'AMERICAN LAGER', 'Origen': 'EEUU',..."
8,lbt_1,Laugar EPA! Euskadiko Pale Ale,2.7,American Pale Ale,American Pale Ale de 5.2% ABV elaborada con lú...,Laugar,165,https://www.labirratorium.com/30978-large_defa...,"{'Estilo': 'APA (AMERICAN PALE ALE)', 'Origen'..."
9,lbt_1,Westmalle Dubbel,2.5,Dubbel,Cerveza Trapista belga con 7% ABV de color mar...,Van Westmalle,82,https://www.labirratorium.com/102-large_defaul...,"{'Estilo': 'DUBBEL', 'Origen': 'Bélgica', '% A..."
